In [ ]:
import os
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from IPython.display import display
import ipywidgets as widgets

def H1(text):
    return widgets.HTML(f"<h1>{text}</h1>")
    
def H2(text):
    return widgets.HTML(f"<h2>{text}</h2>")

def evaluate_all(data_dir):
    plt.close("all")
    display(H1("Spektroskopie"))
    
    data = np.load(os.path.join(data_dir, "DataV42Part1.npy"))
    alphas = data[0]
    betas = data[1]

    refractionIndex = np.sin(np.radians(alphas)) / np.sin(np.radians(betas))
    print("Brechungsindicees n: ", np.round(refractionIndex, 3))
    print("Mittelwert = {0:.3f}".format(np.mean(refractionIndex)))
                              
    angleOfTotalReflection = np.load(os.path.join(data_dir, "DataV42angleOfTotalReflection.npy"))
    refractionIndex_tot = (1 / np.sin(np.radians(angleOfTotalReflection)))
    print("Brechungsindex über Totalreflexion = {0:.3f}".format(refractionIndex_tot))
    
    
    display(H2("Kalibrierung"))
    
    data = np.load(os.path.join(data_dir, "DataV42Part2.npy")).T
    data = pd.DataFrame(data, columns=["x", "lamda", "color"]).astype({"x": float, "lamda": float})

    # Approximate the curve with a polynomial
    def l(x, a, b, c, d):
        return a + x * (b + x * (c + x * d))
    popt, pcov = curve_fit(l, data.x, data.lamda)
    #display(popt)
    
    x = np.linspace(5, 11, 50)
    
    plt.figure()
    plt.plot(x, l(x, *popt))
    plt.plot(data.x, data.lamda, marker="x", ls="")
    plt.grid()
    plt.show()
    
    
    display(H2("Neonspektrum"))
    
    data_neon = np.load(os.path.join(data_dir, "DataV42Part3.npy")).T
    data_neon = pd.DataFrame(data_neon, columns=["x", "color"]).astype({"x": float})
    data_neon["lamda"] = l(data_neon.x, *popt)
    plt.figure()
    plt.plot(x, l(x, *popt))
    for row in data_neon.itertuples():
        plt.axvline(row.x, color='k', lw=.6)
        plt.axhline(row.lamda, color='r', lw=.6)
    plt.plot(data.x, data.lamda, marker="x", ls="")
    plt.grid()
    plt.show()

    display(data_neon)
    
# Interactive student selection
with open("students.txt", "r") as f:
    students = [line.strip("\n") for line in f if not line.startswith("#")]
    
tb = widgets.ToggleButtons(
    options=students,
    description='Name:',
)
display(tb)

output = widgets.Output()
display(output)

@output.capture(clear_output=True)
def on_student_change(change):
    name = change["new"]
    display(H1(f"Auswertung für: {name}"))

    name_escaped = name.replace(" ", "_")
    data_dir = os.path.join("V42", name_escaped)
    
    evaluate_all(data_dir)

tb.observe(on_student_change, names="value")
on_student_change({"new": students[0]})